<a href="https://colab.research.google.com/github/PHaasbroek/Google-Colab/blob/main/Fluid-Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

The aim of this notebook is to simulate a simple numerical domain.

1. Create a laminar, steady, fluid solver for one dimensional pipe flow.
2. Create a RANS turbulence model.
3. Employ the turbulence model in the one dimensional pipe flow.
4. Employ the turbulence model for flow past a backward facing step.
5. Employ the turbulence model for flow past an aerofoil.

# The Navier-Stokes Equations

The incompressible Navier-Stokes (NS) equations for three dimensions are given as:

$$\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} + v \frac{\partial u}{\partial y} + w \frac{\partial u}{\partial z} = -\frac{1}{\rho} \frac{\partial p}{\partial x} + \nu \left(\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} + \frac{\partial^2 u}{\partial z^2}\right)$$

$$\frac{\partial v}{\partial t} + u \frac{\partial v}{\partial x} + v \frac{\partial v}{\partial y} + w \frac{\partial v}{\partial z} = -\frac{1}{\rho} \frac{\partial p}{\partial y} + \nu \left(\frac{\partial^2 v}{\partial x^2} + \frac{\partial^2 v}{\partial y^2} + \frac{\partial^2 v}{\partial z^2}\right)$$

$$\frac{\partial w}{\partial t} + u \frac{\partial w}{\partial x} + v \frac{\partial w}{\partial y} + w \frac{\partial w}{\partial z} = -\frac{1}{\rho} \frac{\partial p}{\partial z} + \nu \left(\frac{\partial^2 w}{\partial x^2} + \frac{\partial^2 w}{\partial y^2} + \frac{\partial^2 w}{\partial z^2}\right)$$


$$\frac{\partial p}{\partial x} + \frac{\partial p}{\partial y} + \frac{\partial p}{\partial y} = 0$$

where:

* $u$ is the velocity in the $x$-direction
* $v$ is the velocity in the $y$-direction
* $w$ is the velocity in the $z$-direction
* $p$ is the pressure
* $\rho$ is the density
* $\nu$ is the kinematic viscosity

# Solution to One-Dimensional Pipe Flow
## Navier-Stokes equations
For the first (1.) case, i.e. one dimensional flow through a pipe, the NS equation is reduced to:
$$ \frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} + v \frac{\partial u}{\partial y} + w \frac{\partial u}{\partial z} = -\frac{1}{\rho} \frac{\partial p}{\partial x} + \nu \left(\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} + \frac{\partial^2 u}{\partial z^2}\right)$$
$$\frac{\partial u}{\partial t} = 0$$

The dimensionless first cell height, $y_1^+$, is defined as
$$y_1^+ = \frac{\rho U_\tau y_1}{\mu},$$
where $\rho$ is the fluid density, $U_\tau$ is the friction velocity, $y_1$ is the first cell height, and $\mu$ is the fluid dynamic viscosity. Friction velocity, $U_\tau$ is given as
$$U_\tau = \sqrt{\frac{\tau_w}{\rho}}, $$ where wall shear stress, $\tau_w$, is given as
$$\tau_w = \frac{1}{2} C_f \rho U^2.$$
Skin friction, $C_f$, is approximated empirically as
$$C_f = 0.079 Re^{-0.25}$$
for internal flow, and as
$$C_f = 0.058 Re^{-0.2}$$
for external flow.


In [5]:
import math

cellNumberHeight = 100 # number of cells through the height of the domain
cellWidth = 1.0 # unit width of cells
firstCellHeight = 8e-5 # [m]
diameter = 0.050 # [m]
meanVelocity = 2.0 # [m/s]
density = 1000.0 # [kg/m^3]
dynamicViscosity = 0.01 # [Pa * s]

Re = diameter * meanVelocity * density / dynamicViscosity
skinFriction = 0.079 * Re**(-0.25)
wallShearStress = 0.5 * skinFriction * density * meanVelocity**2
frictionVelocity = math.sqrt(wallShearStress / density)
dimensionlessFirstCellHeight = density * frictionVelocity * firstCellHeight / dynamicViscosity #

print("Reynolds number: " + str(Re) + " [-]")
print("Skin friction: " + str(skinFriction) + " [-]")
print("Wall shear stress: " + str(wallShearStress) + " [Pa]")
print("Friction velocity: " + str(frictionVelocity) + " [m/s]")
print("Dimensionless first cell height: " + str(dimensionlessFirstCellHeight) + " [-]")

# Make the flow domain



Reynolds number: 10000.0 [-]
Skin friction: 0.0079 [-]
Wall shear stress: 15.8 [Pa]
Friction velocity: 0.12569805089976535 [m/s]
Dimensionless first cell height: 1.005584407198123 [-]


## Mesh Generation
The mesh is created by:
1. Assume the first cell height is as specified.
2. Assume the number of cells are as specified.
3. Assume constant expansion ratio.
3. Calculating the expansion ratio from the given specifications.



In [13]:
sumOfGeometricSeries_f = lambda firstEntry, ratio, numberOfEntries: firstEntry * (1 - ratio**numberOfEntries) / (1 - ratio)
totalHeightZero_f = lambda expansionRatio: sumOfGeometricSeries_f(firstCellHeight, expansionRatio, cellNumberHeight) - diameter

# prompt: Generate  a loop which solves a non linear function using iteration.
def nonLinearSolver(f, x0, tol=1e-2, max_iter=1000):
  """
  Solves a non-linear function using iteration.

  Args:
    f: The function to solve.
    x0: The initial guess.
    tol: The tolerance for convergence.
    max_iter: The maximum number of iterations.

  Returns:
    The solution to the non-linear function.
  """

  for i in range(max_iter):
    x1 = x0 - f(x0) / derivative(f, x0)  # Newton-Raphson method
    print(x1)
    if abs((x1 - x0)/x0) < tol:
      return x1
    x0 = x1


  raise Exception("Non-linear solver did not converge.")

def derivative(f, x):
  """
  Calculates the derivative of a function using the central difference method.

  Args:
    f: The function to differentiate.
    x: The point at which to calculate the derivative.

  Returns:
    The derivative of the function at x.
  """

  h = 1e-6
  return (f(x + h) - f(x - h)) / (2 * h)

nonLinearSolver(totalHeightZero_f, 1.5)

1.4845360824834104
1.4692215092098626
1.4540543826371852
1.4390327904526232
1.4241548016301144
1.4094184618887264
1.3948217884332637
1.380362763831477
1.3660393288457693
1.3518493739907205
1.3377907295283469
1.32386115353284
1.3100583175520166
1.2963797892516313
1.2828230112379124
1.2693852749923635
1.2560636884909384
1.2428551355732376
1.2297562244150093
1.2167632214510098
1.2038719656926913
1.1910777565013517
1.1783752056020012
1.1657580422091023
1.1532188614820622
1.1407488204521465
1.1283373399475223
1.1159720463856313
1.103639711215133
1.0913304503025878
1.0790516306129931
1.066868981222927
1.0550176876623614
1.0441571907234646
1.035712874490341


1.035712874490341